In [1]:
from gurobipy import Model, GRB

In [2]:
# Create a new model
m = Model("farm_boy_expansion")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [3]:
# Decision Variables
# x_n: binary variable, 1 if a supermarket is built at site n, 0 otherwise
# s_n: size of the supermarket at site n in square meters (continuous variable)
x = {n: m.addVar(vtype=GRB.BINARY, name=f"x_{n}") for n in range(1, 16)}
s = {n: m.addVar(vtype=GRB.CONTINUOUS, name=f"s_{n}", lb=0) for n in range(1, 16)}

In [4]:
# Objective Function: Minimize Total Costs
m.setObjective(sum(2000000 * x[n] + 2.00 * s[n] for n in range(1, 16)), GRB.MINIMIZE)

In [5]:
# Constraints
# a) At least 8 supermarkets
m.addConstr(sum(x[n] for n in range(1, 16)) >= 8, "min_supermarkets")

# b) At least 296,000 square meters in total
m.addConstr(sum(s[n] for n in range(1, 16)) >= 296000, "min_total_size")

# c) Maximum size of any supermarket
for n in range(1, 16):
    m.addConstr(s[n] <= 23000 * x[n], f"max_size_{n}")

# Modified constraints based on user input
# 1. Dependency constraint for site 1
m.addConstr(3 * x[1] <= x[7] + x[12] + x[15], "dependency_site_1")

# 2. Mutual exclusivity of sites 3 and 13
m.addConstr(x[3] + x[13] <= 1, "mutual_exclusivity_3_13")

# 3. Size ratio constraints
# Sum of sizes at sites 6-10 must be at least 1.2 times the sum at sites 1-5
m.addConstr(sum(s[n] for n in range(6, 11)) >= 1.2 * sum(s[n] for n in range(1, 6)), "size_ratio_6_10_to_1_5")
# Sum of sizes at sites 6-10 must be less than 3.4 times the sum at sites 11-15
m.addConstr(sum(s[n] for n in range(6, 11)) <= 3.4 * sum(s[n] for n in range(11, 16)), "size_ratio_6_10_to_11_15")

# Other constraints based on initial problem setup
# Number of supermarkets at certain sites must equal 4
m.addConstr(x[3] + x[5] + x[7] + x[9] + x[11] + x[13] == 4, "exact_four_supermarkets")

# Number of supermarkets at certain sites must be at least 2
m.addConstr(x[4] + x[6] + x[8] + x[10] + x[14] >= 2, "at_least_two_supermarkets")

# Linking supermarkets at locations 4 and 10
m.addConstr(x[4] <= x[10], "link_4_to_10")

<gurobi.Constr *Awaiting Model Update*>

In [6]:
# Optimize model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 24 rows, 30 columns and 99 nonzeros
Model fingerprint: 0x9b6be9c4
Variable types: 15 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [2e+00, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+05]
Found heuristic solution: objective 2.659200e+07
Presolve removed 8 rows and 6 columns
Presolve time: 0.00s
Presolved: 16 rows, 24 columns, 55 nonzeros
Variable types: 13 continuous, 11 integer (11 binary)

Root relaxation: objective 2.633113e+07, 29 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2.6331e+07    0    1 2.6592e+07 2.6331e+07  0.98%     - 

In [7]:
# Print solution
if m.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for n in range(1, 16):
        print(f"Location {n}: Built = {x[n].X}, Size = {s[n].X if x[n].X > 0 else 0:.0f} sqm")
else:
    print("No optimal solution found.")

Optimal solution found:
Location 1: Built = 1.0, Size = 20000 sqm
Location 2: Built = 1.0, Size = 23000 sqm
Location 3: Built = 1.0, Size = 23000 sqm
Location 4: Built = 1.0, Size = 23000 sqm
Location 5: Built = 0.0, Size = 0 sqm
Location 6: Built = 1.0, Size = 23000 sqm
Location 7: Built = 1.0, Size = 23000 sqm
Location 8: Built = 1.0, Size = 23000 sqm
Location 9: Built = 1.0, Size = 23000 sqm
Location 10: Built = 1.0, Size = 23000 sqm
Location 11: Built = 1.0, Size = 23000 sqm
Location 12: Built = 1.0, Size = 23000 sqm
Location 13: Built = 0.0, Size = 0 sqm
Location 14: Built = 1.0, Size = 23000 sqm
Location 15: Built = 1.0, Size = 23000 sqm
